In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as f 
from torch.autograd import Variable
torch.backends.cudnn.deterministic = True
import pandas as pd
import numpy as np
import torch.utils.data as data_utils
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random 
from matplotlib import pyplot as plt
from sklearn import decomposition
from sklearn import manifold
from scipy import stats
from itertools import cycle
import sys
import utilities as ut
from Network import Net
import Network as nn
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True



results = []
num_classes = 2
learning_rate = 0.005
samples = 3000
epsilon=0
#for epsilon in [0.1, 0.05, 0.025, 0.15]:
for batch_size in [256]:
    for hidden_size in [100]:
        for aux_loss_activated in [True]:
            for EPS1 in [0.025]:
                for n in [5000, 10000, 50000]:
                    for opt in [1]:
                        for t in range(10):
                            
                            train_dataset, test_dataset = ut.load_files(dataset=1)
                            input_size = train_dataset.shape[1]-1
                            train_dataset, test_dataset = ut.delete_outliers(train_dataset, test_dataset)
                                                        
                            if n < 50000:
                                train_dataset = ut.down_sampling(train_dataset)
                                train_dataset = train_dataset.sample(n)
                            else: 
                                trainig_dataset_a = train_dataset[train_dataset.label=='ClassA']
                                print('shape: ', trainig_dataset_a.shape[0])
                                n2 = n - trainig_dataset_a.shape[0]
                                print('clase no RR Lrae', n2)
                                trainig_dataset_b = train_dataset[~(train_dataset.label=='ClassA')].sample(n2)
                                train_dataset = pd.concat([trainig_dataset_a, trainig_dataset_b])                           
                                
                            train_dataset = ut.sort_columns(train_dataset)
                            test_dataset = ut.sort_columns(test_dataset)
                            
                            data_prior = ut.generate_samples_2D(samples, train_dataset, distribution='gaussian')
                            if train_dataset[train_dataset.label=='ClassB'].shape[0] >= samples:
                                samples_prior = samples 
                            else: 
                                samples_prior = train_dataset[train_dataset.label=='ClassB'].shape[0] 

                            data_prior = pd.concat([data_prior, train_dataset[train_dataset.label=='ClassB'].sample(samples_prior)])


                            train_dataset, test_dataset, data_prior = ut.normalize(train_dataset, test_dataset, data_prior)


                            train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2)

                            train_dataset_prior, val_dataset_prior = train_test_split(data_prior, test_size=0.2)
                            _, _, train_target_prior, train_loader_prior = ut.get_tensors(train_dataset_prior, batch_size)
                            _, _, val_target_prior, val_loader_prior     = ut.get_tensors(val_dataset_prior, batch_size)
                            _, _, train_target, train_loader             = ut.get_tensors(train_dataset, batch_size)
                            _, _, val_target, val_loader                 = ut.get_tensors(val_dataset_prior, batch_size)
                            _, _, test_target, test_loader               = ut.get_tensors(test_dataset, batch_size)

                            net = Net(input_size, hidden_size, hidden_size, num_classes)
                            net.cuda()

                            hist_val, hist_train, _ = nn.train(net, train_loader, train_loader_prior, val_loader, test_loader,
                            EPS1, learning_rate, input_size, aux_loss_activated=aux_loss_activated)

                            acc_test, recall_test, f1_test  = nn.get_results(net, test_loader, input_size)
                            roc_test = nn.get_roc_curve(net, test_loader, input_size)


                            results.append([acc_train, acc_test,recall_train, recall_test, f1_train, f1_test, roc_train, roc_test, epsilon, batch_size, hidden_size, aux_loss_activated, EPS1, n, opt])
                            pd.DataFrame(results, columns=[ 'acc_test', 'recall_test', 'f1_test', 
                                                            'roc_test', 'epsilon', 'batch_size', 'hidden_size',
                                 'aux_loss_activated', 'EPS1', 'n', 'opt']).to_csv('02-08-2022-2d_gaussian-RRLyrae_sc_RRab.csv')


/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/franciscoperez/anacon

Freq1_harmonics_rel_phase_0
Freq2_harmonics_rel_phase_0
Freq3_harmonics_rel_phase_0
downsampling
28243
345650
done downsampling
           Amplitude     PeriodLS
Amplitude   0.014690     1.572135
PeriodLS    1.572135  1406.304913
           Amplitude       PeriodLS
Amplitude   0.002802      -4.373519
PeriodLS   -4.373519  340850.908754
(2992, 61)
(2992, 61)
____get_tensor_function____
shape tensor:  torch.Size([4360, 60])
____get_tensor_function____
shape tensor:  torch.Size([1091, 60])
____get_tensor_function____
shape tensor:  torch.Size([4000, 60])
____get_tensor_function____
shape tensor:  torch.Size([1091, 60])
____get_tensor_function____


/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


shape tensor:  torch.Size([27293, 60])
{'fc1.weight': Parameter containing:
tensor([[-0.1216, -0.0253, -0.0620,  ..., -0.0635, -0.0172, -0.0142],
        [-0.0009,  0.0740,  0.0414,  ..., -0.0388,  0.0627, -0.0103],
        [-0.1228,  0.0421,  0.1236,  ..., -0.0735, -0.0756, -0.0063],
        ...,
        [-0.0505,  0.1162, -0.0782,  ..., -0.0003,  0.1011, -0.0680],
        [-0.0534,  0.1165,  0.0877,  ..., -0.0198,  0.0507, -0.0838],
        [-0.0666, -0.0228, -0.0282,  ..., -0.0681,  0.0579, -0.0940]],
       device='cuda:0', requires_grad=True), 'fc1.bias': Parameter containing:
tensor([-0.0407, -0.0272, -0.0856, -0.0314, -0.0396,  0.0784,  0.0362,  0.0466,
         0.0285, -0.0747, -0.0543, -0.0546, -0.0488,  0.0007,  0.1109, -0.0015,
        -0.0817, -0.0724,  0.0120, -0.1037, -0.0560, -0.0083, -0.0313,  0.0688,
        -0.1168,  0.0401, -0.0505,  0.0337, -0.0443, -0.0740,  0.0815, -0.0001,
        -0.0540, -0.0436,  0.1025,  0.0155, -0.1064,  0.1092,  0.0476, -0.0823,
        -0.

training: epoch:  3  loss:  5.473267585039139 -- aux loss:  7.285067856311798
training: epoch:  4  loss:  5.297472983598709 -- aux loss:  6.383735477924347
training: epoch:  5  loss:  5.2444963455200195 -- aux loss:  6.1079767644405365
training: epoch:  6  loss:  5.222329020500183 -- aux loss:  6.39533606171608
training: epoch:  7  loss:  5.202525824308395 -- aux loss:  5.970928937196732
training: epoch:  8  loss:  5.188048481941223 -- aux loss:  6.024306267499924
training: epoch:  9  loss:  5.182099491357803 -- aux loss:  5.899259597063065
training: epoch:  10  loss:  5.16144335269928 -- aux loss:  5.70618548989296
the_last_loss:  0.0
running_loss_val:  0.3164120018482208
validating: epoch:  10  loss:  0.3164120018482208
The current loss: 0.3164120018482208
the_last_loss: 0.0
trigger times: 1
sum mask2 - L1:  tensor(4799, device='cuda:0')
sum mask2 - L2:  tensor(7553, device='cuda:0')
sum mask2 - L3:  tensor(151, device='cuda:0')
sum mask1 - L1 (aux):  tensor(1201, device='cuda:0')
su

...

training: epoch:  1300  loss:  5.017095983028412 -- aux loss:  5.638994455337524
the_last_loss:  0.31326213479042053
running_loss_val:  0.3132643401622772
validating: epoch:  1300  loss:  0.3132643401622772
The current loss: 0.3132643401622772
the_last_loss: 0.31326213479042053
trigger times: 1
sum mask2 - L1:  tensor(4799, device='cuda:0')
sum mask2 - L2:  tensor(7553, device='cuda:0')
sum mask2 - L3:  tensor(151, device='cuda:0')
sum mask1 - L1 (aux):  tensor(1201, device='cuda:0')
sum mask1 - L2 (aux):  tensor(2447, device='cuda:0')
sum mask1 - L3 (aux):  tensor(49, device='cuda:0')
training: epoch:  1301  loss:  5.019706815481186 -- aux loss:  5.64199885725975
training: epoch:  1302  loss:  5.01645690202713 -- aux loss:  5.641834169626236
training: epoch:  1303  loss:  5.016111731529236 -- aux loss:  5.638849467039108
training: epoch:  1304  loss:  5.016162425279617 -- aux loss:  5.639380544424057
training: epoch:  1305  loss:  5.0161091685295105 -- aux loss:  5.638878703117371
tra

training: epoch:  1362  loss:  5.016094833612442 -- aux loss:  5.63871306180954
training: epoch:  1363  loss:  5.016112506389618 -- aux loss:  5.640413522720337
training: epoch:  1364  loss:  5.0160976350307465 -- aux loss:  5.638726085424423
training: epoch:  1365  loss:  5.016097456216812 -- aux loss:  5.638711541891098
training: epoch:  1366  loss:  5.016094982624054 -- aux loss:  5.638720065355301
training: epoch:  1367  loss:  5.016093194484711 -- aux loss:  5.63871157169342
training: epoch:  1368  loss:  5.016096085309982 -- aux loss:  5.638711869716644
training: epoch:  1369  loss:  5.016094774007797 -- aux loss:  5.638711363077164
training: epoch:  1370  loss:  5.0160965621471405 -- aux loss:  5.638715893030167
the_last_loss:  0.3132617175579071
running_loss_val:  0.3132617771625519
validating: epoch:  1370  loss:  0.3132617771625519
The current loss: 0.3132617771625519
the_last_loss: 0.3132617175579071
trigger times: 1
sum mask2 - L1:  tensor(4799, device='cuda:0')
sum mask2 -

training: epoch:  1428  loss:  5.01617032289505 -- aux loss:  5.638748079538345
training: epoch:  1429  loss:  5.016093462705612 -- aux loss:  5.668659687042236
training: epoch:  1430  loss:  5.01609680056572 -- aux loss:  5.638785034418106
the_last_loss:  0.31326162815093994
running_loss_val:  0.31326162815093994
validating: epoch:  1430  loss:  0.31326162815093994
The current loss: 0.31326162815093994
the_last_loss: 0.31326162815093994
trigger times: 1
sum mask2 - L1:  tensor(4799, device='cuda:0')
sum mask2 - L2:  tensor(7553, device='cuda:0')
sum mask2 - L3:  tensor(151, device='cuda:0')
sum mask1 - L1 (aux):  tensor(1201, device='cuda:0')
sum mask1 - L2 (aux):  tensor(2447, device='cuda:0')
sum mask1 - L3 (aux):  tensor(49, device='cuda:0')
training: epoch:  1431  loss:  5.016101539134979 -- aux loss:  5.63871094584465
training: epoch:  1432  loss:  5.016096621751785 -- aux loss:  5.638713389635086
training: epoch:  1433  loss:  5.01844710111618 -- aux loss:  5.638715624809265
tra

training: epoch:  1492  loss:  5.016098439693451 -- aux loss:  5.638716757297516
training: epoch:  1493  loss:  5.016094148159027 -- aux loss:  5.708851397037506
training: epoch:  1494  loss:  5.016092956066132 -- aux loss:  5.638710469007492
training: epoch:  1495  loss:  5.016106337308884 -- aux loss:  5.638709455728531
training: epoch:  1496  loss:  5.016092866659164 -- aux loss:  5.638709723949432
training: epoch:  1497  loss:  5.016093492507935 -- aux loss:  5.638710469007492
training: epoch:  1498  loss:  5.016096293926239 -- aux loss:  5.638813406229019
training: epoch:  1499  loss:  5.016093045473099 -- aux loss:  5.638712078332901
training: epoch:  1500  loss:  5.016095668077469 -- aux loss:  5.638709872961044
the_last_loss:  0.3132617771625519
running_loss_val:  0.3133096694946289
validating: epoch:  1500  loss:  0.3133096694946289
The current loss: 0.3133096694946289
the_last_loss: 0.3132617771625519
trigger times: 2
sum mask2 - L1:  tensor(4799, device='cuda:0')
sum mask2 -

NameError: name 'fail' is not defined